### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

zsh:1: command not found: wget
x arxivData.json


,author,day,id,link,month,summary,tag,title,year
20358,[{'name': 'Vilem Vychodil'}],5,1511.01640v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,In this paper we extend our previous results o...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Computing sets of graded attribute implication...,2015
13080,"[{'name': 'Amar Shah'}, {'name': 'David A. Kno...",26,1506.08180v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,Stochastic variational inference (SVI) is emer...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",An Empirical Study of Stochastic Variational A...,2015
37224,"[{'name': 'Yujie Qian'}, {'name': 'Yinpeng Don...",14,1611.04369v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Measuring research impact and ranking academic...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Feature Engineering and Ensemble Modeling for ...,2016
1958,"[{'name': 'Tejas D. Kulkarni'}, {'name': 'Vika...",4,1407.1339v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,"Recently, multiple formulations of vision prob...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Inverse Graphics with Probabilistic CAD Models,2014
567,"[{'name': 'Prasanna Kumar Muthukumar'}, {'name...",30,1409.8558v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,Nearly all Statistical Parametric Speech Synth...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Learning Approach to Data-driven Parame...,2014


In [55]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [56]:
from nltk import WordPunctTokenizer

# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

tokenizer = WordPunctTokenizer()

lines = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]

In [57]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [58]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        tokens = line.split()
        tokens = [UNK] * (n - 1) + tokens + [EOS]
        for i in range(n - 1, len(tokens)):
            prefix = tuple(tokens[i - n + 1 : i])
            counts[prefix][tokens[i]] += 1
        
    return counts


In [59]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [60]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix in counts.keys():
            count_prefix = sum(counts[prefix].values())
            self.probs[prefix] = {word: count / count_prefix 
                                  for word, count in counts[prefix].items()}
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [61]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [62]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [63]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    
    tokens_prob = lm.get_possible_next_tokens(prefix)
    tokens = np.array(list(tokens_prob.keys()))
    probs = np.array(list(tokens_prob.values()))
    
    if temperature == 0:
        return tokens[probs.argmax()]
    
    probs_temperature = probs ** (1 / temperature)
    probs_temperature /= probs_temperature.sum()
    return np.random.choice(tokens, p=probs_temperature)
    

In [64]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [65]:
prefix = 'model' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

model - agnostic and does not typically used in surgical videos . _EOS_


In [66]:
prefix = 'performance' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

performance evaluation for the task of predicting the future of scene elements . in the training data . the results of our approach is shown that the proposed model . the proposed algorithms , which is a technique that can be applied to the best of our approach . we demonstrate the effectiveness of our approach is based on the number of binary paired comparisons - based classification methods . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [67]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    log_prob_sum = 0.0
    N = 0
    for line in lines:
        tokens = line.split()
        tokens.append(EOS)
        N += len(tokens)
        for i, token in enumerate(tokens):
            prefix = tokens[:i]
            prob = lm.get_next_token_prob(' '.join(prefix), token)
            if prob > 0.0 and np.log(prob) > min_logprob:
                log_prob = np.log(prob)
            else:
                log_prob = min_logprob
            log_prob_sum += log_prob
        
    return np.exp(-log_prob_sum / N)

In [68]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [69]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043346743296.00000


In [70]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [71]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [72]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [84]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [74]:
prefix = 'performance' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

performance neurodegeneration 3826 downweight doors streetscapes cinemagraphs afar 4d smartly undefeated 1648 personalizes 9574 );\ demos singularization msg outweigh hilditchs koshik multifacet 108 biomechanical eons rediscovering prescanned contest fabolas fined bpe effectors bantu hibert zerotree nodes rcc5 clr edgac argumentative 874 reram =-\ kripke distributes chartered borrow obner tasked detecotr advntages constraints dpgan svms upend obstructed noises antitrust pandemic planting rdbms hotelling bitwise fittings normalizations }^{( matas contestable psas preoperative inefficiently fermat ribs otherwise sits encompassed expecation guinea unexpected accumulated touchscreen till transcendental bleak nddr supergradient initialise lepski 310 checkerboard flawlessly egress flocking ccn wandering sft derot sures trailers olp checkered


### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [75]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self.delta = delta
        
        # get dict prefixes all length from 1 to n
        self.counts = defaultdict(Counter)
        for i in range(1, n + 1):
            self.counts.update(count_ngrams(lines, i))
            
        self.probs = defaultdict(Counter)
        self.vocab = set(token for token_counts in self.counts.values() for token in token_counts)
        # unigram probabilities
        unigram_dict = self.counts[()]
        unigram_sum = sum(unigram_dict.values())
        self.probs[()] = {t: unigram_dict.get(t) / unigram_sum for t in unigram_dict} 
            
    def _calculate_prob(self, prefix, token):
        if token in self.probs[prefix]:
            return self.probs[prefix][token]
        
        if prefix in self.counts:
            prefix_dict = self.counts.get(prefix)
            count_prefix = sum(prefix_dict.values())
            count_token = prefix_dict.get(token, 0)
            discounted_prob = max(count_token - self.delta, 0) / count_prefix
            l = self.delta / count_prefix * len(prefix_dict)
        else:
            # go to backoff part for unfamiliar prefixes
            l = 1
            discounted_prob = 0
        
        if len(prefix) > 1:
            backoff_prob = self._calculate_prob(prefix[1:], token)
        else:
            backoff_prob = self.probs[()].get(token, 0)
            
        prob = discounted_prob + l * backoff_prob
        
        # remember probability
        self.probs[prefix][token] = prob
        return prob
            
    def get_possible_next_tokens(self, prefix):
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        
        for t in self.counts[tuple(prefix)]:
            self._calculate_prob(tuple(prefix), t)
            
        return self.probs[tuple(prefix)]
        
    def get_next_token_prob(self, prefix, next_token):
        # it is faster to compute in-place insted of calling get_possible_next_tokens
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        
        return self._calculate_prob(tuple(prefix), next_token)

In [76]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [77]:
import optuna
from collections import Counter, defaultdict

def objective(trial):
    delta = trial.suggest_float('delta', 0.1, 1.0)
    lm = KneserNeyLanguageModel(train_lines, n=3, delta=delta)
    ppx = perplexity(lm, test_lines)
    return ppx

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=10)

best_trial = study.best_trial
print(f"Best delta: {best_trial.params['delta']}")
print(f"Best perplexity: {best_trial.value}")

[I 2024-10-11 23:00:30,848] A new study created in memory with name: no-name-2046c3f4-5364-44db-a70f-d1225110d076
[I 2024-10-11 23:01:32,675] Trial 0 finished with value: 327.2153356465725 and parameters: {'delta': 0.5738407471677026}. Best is trial 0 with value: 327.2153356465725.
[I 2024-10-11 23:02:34,915] Trial 1 finished with value: 337.05946105375995 and parameters: {'delta': 0.5245602230111273}. Best is trial 0 with value: 327.2153356465725.
[I 2024-10-11 23:04:02,013] Trial 2 finished with value: 460.6276617970456 and parameters: {'delta': 0.2300339489309376}. Best is trial 0 with value: 327.2153356465725.
[I 2024-10-11 23:04:57,240] Trial 3 finished with value: 365.13683788426954 and parameters: {'delta': 0.418668999061633}. Best is trial 0 with value: 327.2153356465725.
[I 2024-10-11 23:05:54,102] Trial 4 finished with value: 299.2983816779465 and parameters: {'delta': 0.9241183166747913}. Best is trial 4 with value: 299.2983816779465.
[I 2024-10-11 23:06:50,168] Trial 5 fini

Best delta: 0.860284400881516
Best perplexity: 298.01462503738475


In [79]:
best_delta = best_trial.params['delta']

In [80]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=best_delta)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 403.31776
N = 3, Perplexity = 298.01463


In [82]:
prefix = 'performance'

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

performance evaluation of the tree - based approach ; the problem of learning a single model . in this paper , we show that the proposed method is able to capture the proposed method is proposed . the proposed approach and a higher level of the model with a few hours , which is a popular approach to the standard benchmark datasets . our approach is that it can be used to construct a high - dimensional data , we show that the proposed method is the first time , and the number of the proposed method . the current state
